In [1]:

%load_ext autoreload
%autoreload 2
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
import numpy as np
import torch

from transformers import pipeline

/home/setup/miniconda3/envs/scr/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from transformers import GPT2Config, GPT2Model,PreTrainedModel,PretrainedConfig
from MoEWrapper import MoEWrapper
cfg=PretrainedConfig()
cfg=GPT2Config()

moe=MoEWrapper(cfg)

model_name_list =[ 
    "llm-jp/llm-jp-13b-instruct-full-jaster-v1.0",
    "llm-jp/llm-jp-13b-instruct-full-dolly_en-dolly_ja-ichikara_003_001-oasst_en-oasst_ja-v1.1",
                  ]

tokenizer = AutoTokenizer.from_pretrained(model_name_list[0])
for model_name in model_name_list:
    model = AutoModelForCausalLM.from_pretrained(
        model_name, device_map="auto",
        torch_dtype=torch.float16
    )
    moe.append_model(model)

Loading checkpoint shards: 100%|██████████| 6/6 [00:06<00:00,  1.06s/it]


In [6]:
max_new_tokens=10
repetition_penalty=2.0
pipe=pipeline("text-generation",model=moe,
              tokenizer=tokenizer,
     max_new_tokens=max_new_tokens,
                      repetition_penalty=repetition_penalty,
 pad_token_id=50256,
              )



The model 'MoEWrapper' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', '

In [7]:
inputs=[
    "hello, I'm John",
    "global warming ",
    "英語: He is a good man. 日本語: ",
    "今日は晴れてるから",
    "ブログを書きました",
    "地球温暖化を防ぐに",
    "ガリレオによるピサの斜塔実験とは",
    "スーパーコンピューターの富岳は",
]
moe.set_flexible_mode()
for text in inputs:
    print("------")
    print("input: ", text)
    print(pipe(text)[0]["generated_text"].replace("\n"," "))

/home/setup/miniconda3/envs/scr/lib/python3.11/site-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


------
input:  hello, I'm John
model 0 will be used
ppl array:  [ 9.34375   11.3984375]
hello, I'm John." 
------
input:  global warming 
model 0 will be used
ppl array:  [1051. 2414.]
global warming ([@B10]). 
------
input:  英語: He is a good man. 日本語: 
model 1 will be used
ppl array:  [23.15625  21.640625]
英語: He is a good man. 日本語: 彼は良い人です。
------
input:  今日は晴れてるから
model 1 will be used
ppl array:  [86.5625 77.625 ]
今日は晴れてるから、洗濯物がよく乾くわ～。
------
input:  ブログを書きました
model 1 will be used
ppl array:  [46.78125  30.984375]
ブログを書きました。  #アメブロ
------
input:  地球温暖化を防ぐに
model 1 will be used
ppl array:  [41.9375 38.9375]
地球温暖化を防ぐにせよ、経済成長の鈍りによる不況から
------
input:  ガリレオによるピサの斜塔実験とは
model 1 will be used
ppl array:  [38.34375  29.515625]
ガリレオによるピサの斜塔実験とは？
------
input:  スーパーコンピューターの富岳は
model 1 will be used
ppl array:  [86.9375 72.625 ]
スーパーコンピューターの富岳は、2014年に開発され


In [8]:
for text in inputs:
    print("------")
    print("input: ", text)
    for model_id in range(len(model_name_list)):
        moe.fix_model(model_id)
        print(f"model id {model_id}: ",pipe(text)[0]["generated_text"].replace("\n"," "))

------
input:  hello, I'm John
model id 0:  hello, I'm John." 
model id 1:  hello, I'm John."  There are many variations on
------
input:  global warming 
model id 0:  global warming ([@B10]). 
model id 1:  global warming ([@bib12]).
------
input:  英語: He is a good man. 日本語: 
model id 0:  英語: He is a good man. 日本語: 彼は良い人です。
model id 1:  英語: He is a good man. 日本語: 彼は良い人です。
------
input:  今日は晴れてるから
model id 0:  今日は晴れてるから、お散歩に行こうね。
model id 1:  今日は晴れてるから、洗濯物がよく乾くわ～。
------
input:  ブログを書きました
model id 0:  ブログを書きました。
model id 1:  ブログを書きました。  #アメブロ
------
input:  地球温暖化を防ぐに
model id 0:  地球温暖化を防ぐに.
model id 1:  地球温暖化を防ぐにせよ、経済成長の鈍りによる不況から
------
input:  ガリレオによるピサの斜塔実験とは
model id 0:  ガリレオによるピサの斜塔実験とは？
model id 1:  ガリレオによるピサの斜塔実験とは？
------
input:  スーパーコンピューターの富岳は
model id 0:  スーパーコンピューターの富岳は、2018年に世界で初めて
model id 1:  スーパーコンピューターの富岳は、2014年に開発され


In [22]:
instruction="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。"
text_list=[
"前提と仮説の関係をyes、no、unknown、undefの中から回答してください。それ以外には何も含めないことを厳守してください。\n\n制約：\n- 前提が仮説を含意する場合はyesと出力\n- 前提が仮説の否定を含意する場合はnoと出力\n- 前提が仮説を含意せず、その否定も含意しない場合はunknownと出力\n- 与えられた情報のみからは判断ができない場合はundefと出力\n前提：あっちの学校は校則が厳しいことで有名で、こっちの学校は自由な校風を売りにしている。\n仮説：あっちの学校は校則が厳しいことで有名なので、こっちの学校は自由な校風を売りにしている。",
"与えられた文章から固有表現で書かれたターゲットの名前を抽出し、それに対する極性をpositive、neutral、negativeの中から選択して下さい。固有表現で書かれたターゲットの名前と、それに対する極性（positive、neutral、negativeのいずれか）のペアをスペース（ ）で区切って出力し、それ以外には何も含めないことを厳守してください。答えが複数の場合、改行で繋げてください。ただし、ターゲットは固有表現である市場、市況、会社/法人、グループ、会社内の部門、事業部、事業領域、製品、サービスの名称などを指すこととします。\n文章：建設事業受注高は、前連結会計年度と同水準で推移し、前連結会計年度比3.8％減の１兆7,283億円（前連結会計年度は１兆7,958億円）となった"
"前提と仮説の関係をentailment、contradiction、neutralの中から回答してください。それ以外には何も含めないことを厳守してください。\n\n制約：\n- 前提から仮説が、時間関係上導出可能である場合はentailmentと出力\n- 前提と仮説が両立しえない場合はcontradictionと出力\n- そのいずれでもない場合はneutralと出力\n前提：ボブは選挙に撤回を表明している。\n仮説：ボブは現在選挙に撤回を表明している。"
]
#text_list=[instruction+"\n##指示:\n"+i+"\n##応答:\n" for i in text_list]
text_list=[instruction+"\n##指示:\n"+i+"\n##応答:\n" for i in text_list]
moe.set_flexible_mode()

moe.fix_model(0)
for text in text_list:
    print("------")
    print("input: ", text)
    out=pipe(text)[0]["generated_text"]
    out=out[len(text):]
    print("out")
    print(out.replace("\n"," "))

------
input:  以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。
##指示:
前提と仮説の関係をyes、no、unknown、undefの中から回答してください。それ以外には何も含めないことを厳守してください。

制約：
- 前提が仮説を含意する場合はyesと出力
- 前提が仮説の否定を含意する場合はnoと出力
- 前提が仮説を含意せず、その否定も含意しない場合はunknownと出力
- 与えられた情報のみからは判断ができない場合はundefと出力
前提：あっちの学校は校則が厳しいことで有名で、こっちの学校は自由な校風を売りにしている。
仮説：あっちの学校は校則が厳しいことで有名なので、こっちの学校は自由な校風を売りにしている。
##応答:

out
neutral
------
input:  以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。
##指示:
与えられた文章から固有表現で書かれたターゲットの名前を抽出し、それに対する極性をpositive、neutral、negativeの中から選択して下さい。固有表現で書かれたターゲットの名前と、それに対する極性（positive、neutral、negativeのいずれか）のペアをスペース（ ）で区切って出力し、それ以外には何も含めないことを厳守してください。答えが複数の場合、改行で繋げてください。ただし、ターゲットは固有表現である市場、市況、会社/法人、グループ、会社内の部門、事業部、事業領域、製品、サービスの名称などを指すこととします。
文章：建設事業受注高は、前連結会計年度と同水準で推移し、前連結会計年度比3.8％減の１兆7,283億円（前連結会計年度は１兆7,958億円）となった前提と仮説の関係をentailment、contradiction、neutralの中から回答してください。それ以外には何も含めないことを厳守してください。

制約：
- 前提から仮説が、時間関係上導出可能である場合はentailmentと出力
- 前提と仮説が両立しえない場合はcontradictionと出力
- そのいずれでもない場合はneutralと出力
前提：ボブは選挙に撤回を表明している。
仮説：ボブは現在